<a href="https://colab.research.google.com/github/Limnackhoon/github-slideshow/blob/master/cocodataset01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#json파일 생성
"info": {
    #2017년 코코 데이터셋
    "description": "COCO 2017 Dataset",
    #cocodataset의 웹 페이지 주소
    "url": "http://cocodataset.org",
    #버전 : 1.0
    "version": "1.0",
    #2017년도
    "year": 2017,
    #COCO Consortium으로부터 기여됨
    "contributor": "COCO Consortium",
    #2017년9월1일에 데이터생성됨
    "date_created": "2017/09/01"
  }
  ...
#image에 대한 bbox와 segmentation 정보
"area": 1037.7818999999995,
      "iscrowd": 0,
      "image_id": 397133,
      "bbox": [
        0,
        262.81,
        62.16,
        36.77
      ],
      "category_id": 1,
      "id": 1218137
    }

"segmentation": [
        [
          292.37,
          425.1,
          340.6,
          373.86,
          347.63,
          256.31,
          198.93,
          240.24,
          4.02,
          311.57,
          1,
          427,
          291.36,
          427
        ]
      ]

In [ ]:
#Data준비
#다운 받은 데이터를 다음과 같은 경로로 저장
#2017 Train images와 2017 Train/Val annotations 데이터셋 사용
root -- images      -- train2017
     |              |- val2017
     |              |- test2017
     | 
     -- anotations  -- instances_train2017.json
                    |- instances_val2017.json  * minival        

In [ ]:
#Data Loader
#필요한 운영체제와 라이브러리 import
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
from transform import transform_COCO
from PIL import Image
import matplotlib
from matplotlib.patches import Rectangle

In [ ]:
#Data Loader Class를 만들어줌
class COCO_Dataset(Dataset):
  #경로는 D:\Data\coco, 설정이름은 val2017,
  #RAIN기준으로 문자열을 나누기
  #경로 오류가 나지 않도록 root_dir 작성 주의
	def __init__(self, root_dir='D:\Data\coco', set_name='val2017', split='TRAIN'):
	        #init method를 사용해서 초기화
	         super().__init__()
	        self.root_dir = root_dir
	        self.set_name = set_name
          #join으로 D:\Data\coco+ annotations, instance_,val2017+json을 합쳐서 하나의 문자열로 바꾸어 반환
	        self.coco = COCO(os.path.join(self.root_dir, 'annotations', 'instances_' + self.set_name + '.json'))

In [ ]:
 #COCO안에 들어있는 약 80개의 classes중 모든 이미지들을 self.coco.getImgIds()안에 넣어줌
 whole_image_ids = self.coco.getImgIds() 
	        #self.image_ids를 list에 담아줌
	        self.image_ids = []
	
	        #라벨이 없는 이미지들을 제거하고 train이나 test에 이용하도록 list에 담아줌
	        self.no_anno_list = []
	        #idx를 전체 이미지 id만큼 반복
	        for idx in whole_image_ids:
              #이미지의 아이디가 idx인 annotations_ids로 불러옴 / iscrowd = 0?
	            annotations_ids = self.coco.getAnnIds(imgIds=idx, iscrowd=False)
              #len으로 리스트의 크기를 알기 위함, 주석이 있는 이미지의 리스트 
              #annotations_ids의 리스트의 크기가 0과 값이 동일하면
	            if len(annotations_ids) == 0:
                  #idx에.no_anno_list자료형을 추가
	                self.no_anno_list.append(idx)
	            else:
                  #그 외의 경우 image_ids를 추가(image_id = 397133?)
	                self.image_ids.append(idx)
	        #클래스의 정보를 읽음
	        self.load_classes()
          # split하여 리스트의 형식으로 만들어줌
	        self.split = split

In [1]:
#더블언더바__getitem__method 사용
#getitem function에서 하나의 image를 열고 bounding box를 읽음
def __getitem__(self, idx):
        #시각화
        visualize = True
        #이미지의 넓이,높이를 취득하기위함
        image, (w, h) = self.load_image(idx)
        annotation = self.load_annotations(idx)
        #32-bit으로 슬라이싱
        boxes = torch.FloatTensor(annotation[:, :4])
        #64-bit integer로 슬라이싱
        labels = torch.LongTensor(annotation[:, 4])

        #라벨에 지정된 이미지가 없으면
        if labels.nelement() == 0:  
            #보여줌
            visualize = True
        #data augmentation
        image, boxes, labels, segmentations = transform_COCO(image, boxes, labels, self.split)
        #image, boxes, labels 값들을 돌려줌
        return image, boxes, labels

In [ ]:
#coco라이브러리를 이용하여 이미지 정보 읽어오기
def load_image(self, image_index):
        #이미지 정보 확인
        image_info = self.coco.loadImgs(self.image_ids[image_index])[0]
        #path를 이용해서 파일탐색경로설정
        path = os.path.join(self.root_dir, 'images', self.set_name, image_info['file_name'])
        #이미지의 경로를 열어서 RGB로 변환한 값을 이미지에 저장
        image = Image.open(path).convert('RGB')
        #이미지의 넓이와 높이 정보를 반환
        return image, (image_info['width'], image_info['height'])

In [ ]:
#coco라이브러리를 이용하여 annotations정보 읽기
def load_annotations(self, image_index):
        # 
        annotations_ids = self.coco.getAnnIds(imgIds=self.image_ids[image_index], iscrowd=False)
        #np.zeros= 0으로 채워진 지정된 모양과 유형의 새 배열을 반환
        #(0,0,0,0,0)형태로 출력
        annotations = np.zeros((0, 5))

        #annotations_ids의 리스트의 크기가 0과 값이 동일하면
        if len(annotations_ids) == 0:
            #annotations를 반환
            return annotations

        #parse annotations
        coco_annotations = self.coco.loadAnns(annotations_ids)
        for idx, a in enumerate(coco_annotations):

            #a의 bbox의 2번 값이 1보다 작거나 a의 bbox의 3번 값이 1보다 작으면
            if a['bbox'][2] < 1 or a['bbox'][3] < 1:
                #계속 진행
                continue
            #1 by 5 - zero matrix 생성
            annotation = np.zeros((1, 5))
            annotation[0, :4] = a['bbox']
            annotation[0, 4] = self.coco_label_to_label(a['category_id'])
            annotations = np.append(annotations, annotation, axis=0)

        #transform from [x, y, w, h] to [x1, y1, x2, y2]
        annotations[:, 2] = annotations[:, 0] + annotations[:, 2]
        annotations[:, 3] = annotations[:, 1] + annotations[:, 3]
        #annotations값 반환
        return annotations

In [ ]:
#train함수에서 train loader를 불러와서 train data들을 이용
train_set = COCO_Dataset()
#train_loader에 저장
train_loader = DataLoader(train_set,
                              batch_size=1,
                              collate_fn=train_set.collate_fn,
                              shuffle=False,
                              num_workers=0,
                              pin_memory=True)

In [ ]:
#for문을 이용해서 data를 읽어옴
for i, (images, boxes, labels) in enumerate(train_loader):
        images = images.cuda()
        boxes = [b.cuda() for b in boxes]
        labels = [l.cuda() for l in labels]